Importación de librerías

In [0]:
import pandas as pd

from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder, LabelBinarizer

In [0]:
 from sklearn.ensemble import RandomForestRegressor

In [0]:
from sklearn.metrics import r2_score

Importación de dataset



In [0]:
df = pd.read_csv("/content/drive/My Drive/Csv4.csv").reset_index(drop=True)

In [81]:
df.head()

,Unnamed: 0,Host Country,Year,Region,Athlete,Sport,Medal_Gold,Medal_Silver,Medal_Bronze,Medals
0,0,ESP,1992,Kuwait,32,7,0,0,0,0
1,1,ESP,1992,Niger,3,1,0,0,0,0
2,2,ESP,1992,Nigeria,55,8,0,7,4,11
3,3,ESP,1992,North Korea,64,12,4,0,6,10
4,4,ESP,1992,Norway,83,17,2,20,1,23


Pequeña limpieza

In [0]:
del df["Unnamed: 0"]

In [0]:
df.drop(columns=["Medal_Gold","Medal_Silver","Medal_Bronze"], inplace=True)

In [84]:
print(df.shape)

df.head(2)

(1365, 6)


,Host Country,Year,Region,Athlete,Sport,Medals
0,ESP,1992,Kuwait,32,7,0
1,ESP,1992,Niger,3,1,0


Transformar datos categóricos a numéricos

In [0]:
#Aplicación de máscara booleana

In [0]:
encoded = df.copy()

In [0]:
maskBool = encoded.dtypes==object

In [0]:
colsCat = encoded.columns[maskBool].tolist()

In [0]:
le = LabelEncoder()

In [0]:
encoded[colsCat] = encoded[colsCat].apply(lambda x: le.fit_transform(x))

In [91]:
encoded.head()

,Host Country,Year,Region,Athlete,Sport,Medals
0,3,1992,96,32,7,0
1,3,1992,132,3,1,0
2,3,1992,133,55,8,11
3,3,1992,134,64,12,10
4,3,1992,135,83,17,23


In [0]:
#Se observa que este procedimiento no es el más adecuado por las grandes cifras que alcanza "Region".
#Se procede a un OneHotEncoder para arreglar esta situación

In [0]:
oneHot = df.copy()

In [0]:
regionEncoder = LabelBinarizer()

regionEncoder.fit(oneHot['Region'])

transformed = regionEncoder.transform(oneHot['Region'])

dfTrans = pd.DataFrame(transformed)

dfDef = pd.concat([encoded, dfTrans], axis=1).drop(columns=['Region'], axis=1)

In [99]:
print(dfDef.shape)

dfDef.head(2)

(1365, 210)


,Host Country,Year,Athlete,Sport,Medals,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204
0,3,1992,32,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,1992,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Elección de características

In [0]:
y = 'Medals'
X = list(dfDef.columns)

Spliteo de la información

In [0]:
X_train, X_test, y_train, y_test = train_test_split(dfDef.drop(columns="Medals"), dfDef["Medals"], test_size=0.2)

Regresión Lineal

In [0]:
model1 = LinearRegression()

In [103]:
model1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Random Forest

In [0]:
model2 = RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [133]:
model2.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

Predicciones

In [0]:
y_pred1 = model1.predict(X_test)

In [155]:
pd.DataFrame(columns = [y_test,y_pred1]).T.head()

,
Medals,
57,74.318329
10,3.296478
0,-0.339355
10,3.408844
27,24.633087


In [156]:
r2_score(y_test, y_pred1)

-2.8096873803011837e+17

In [0]:
y_pred2 = model2.predict(X_test)

In [158]:
pd.DataFrame(columns = [y_test,y_pred2]).T.head()

,
Medals,
57,54.464
10,7.215
0,0.002
10,0.754
27,55.330


In [159]:
r2_score(y_test, y_pred2)

0.80188289898009

Ninguno de estos modelos es escogido por su baja eficacia (la métrica r2 está muy alejada de 0)